In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Optional: For display aesthetics
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
# Load Zillow datasets from the datasets/ folder
nba_advanced = pd.read_csv("datasets/nba_advanced_2010_2024.csv")

nba_advanced.columns = [col.split()[-1] if 'Unnamed' in col else col for col in nba_advanced.columns]

# Optional: clean whitespace
nba_advanced.columns = [col.strip() for col in nba_advanced.columns]

nba_advanced = nba_advanced[[col for col in nba_advanced.columns if "level" not in col]]

# Preview cleaned DataFrame
print(nba_advanced.columns)


nba_pg = pd.read_csv("datasets/nba_per_game_2010_2024.csv")

print(nba_pg.columns)


Index(['Rk', 'Team', 'Age', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS', 'ORtg',
       'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%',
       'Offense Four Factors eFG%', 'Offense Four Factors TOV%',
       'Offense Four Factors ORB%', 'Offense Four Factors FT/FGA',
       'Defense Four Factors eFG%', 'Defense Four Factors TOV%',
       'Defense Four Factors DRB%', 'Defense Four Factors FT/FGA', 'Arena',
       'Attend.', 'Attend./G', 'Season'],
      dtype='object')
Index(['Rk', 'Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P',
       '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'Season'],
      dtype='object')


In [3]:
merged = pd.merge(nba_advanced, nba_pg, on=["Team", "Season"], how="inner")

merged = merged.drop(columns=["Rk_x", "Rk_y"])

cols = merged.columns.tolist()
cols.remove("Season")
cols.remove("Team")

# Reorder
merged = merged[["Season", "Team"] + cols]

# Create a new boolean column indicating playoff status
merged['Playoffs'] = merged['Team'].str.contains(r'\*', regex=True)

# Clean team names (remove asterisk)
merged['Team'] = merged['Team'].str.replace('*', '', regex=False).str.strip()

print(merged.columns)
merged.head()

Index(['Season', 'Team', 'Age', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS',
       'ORtg', 'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%',
       'Offense Four Factors eFG%', 'Offense Four Factors TOV%',
       'Offense Four Factors ORB%', 'Offense Four Factors FT/FGA',
       'Defense Four Factors eFG%', 'Defense Four Factors TOV%',
       'Defense Four Factors DRB%', 'Defense Four Factors FT/FGA', 'Arena',
       'Attend.', 'Attend./G', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA',
       '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Playoffs'],
      dtype='object')


,Season,Team,Age,W,L,PW,PL,MOV,SOS,SRS,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Playoffs
0,2009-10,Orlando Magic,27.9,59,23,61,21,7.49,-0.37,7.12,...,9.9,33.4,43.2,19.7,6.2,5.6,14.1,19.9,102.8,True
1,2009-10,Cleveland Cavaliers,28.0,61,21,59,23,6.52,-0.36,6.17,...,9.6,32.8,42.5,22.4,6.9,5.2,13.9,19.4,102.1,True
2,2009-10,Utah Jazz,25.6,53,29,55,27,5.34,-0.01,5.33,...,10.6,31.6,42.2,26.7,8.2,4.9,15.2,22.7,104.2,True
3,2009-10,San Antonio Spurs,28.7,50,32,55,27,5.09,-0.02,5.07,...,10.8,32.0,42.8,22.3,6.3,4.6,13.6,20.4,101.4,True
4,2009-10,Atlanta Hawks,26.5,53,29,54,28,4.66,-0.22,4.44,...,11.8,29.9,41.7,21.8,7.2,5.0,12.0,19.9,101.7,True


In [9]:
merged.to_csv("nba_team_combined_2010_2024.csv", index=False)

In [4]:
string_cols = ['Season', 'Team', 'Arena']

# Convert the rest to numeric
for col in merged.columns:
    if col not in string_cols:
        merged[col] = pd.to_numeric(merged[col], errors='coerce')

merged.dtypes

Season                          object
Team                            object
Age                            float64
W                              float64
L                              float64
PW                             float64
PL                             float64
MOV                            float64
SOS                            float64
SRS                            float64
ORtg                           float64
DRtg                           float64
NRtg                           float64
Pace                           float64
FTr                            float64
3PAr                           float64
TS%                            float64
Offense Four Factors eFG%      float64
Offense Four Factors TOV%      float64
Offense Four Factors ORB%      float64
Offense Four Factors FT/FGA    float64
Defense Four Factors eFG%      float64
Defense Four Factors TOV%      float64
Defense Four Factors DRB%      float64
Defense Four Factors FT/FGA    float64
Arena                    

In [5]:
merged.head()

,Season,Team,Age,W,L,PW,PL,MOV,SOS,SRS,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Playoffs
0,2009-10,Orlando Magic,27.90,59.00,23.00,61.00,21.00,7.49,-0.37,7.12,...,9.90,33.40,43.20,19.70,6.20,5.60,14.10,19.90,102.80,True
1,2009-10,Cleveland Cavaliers,28.00,61.00,21.00,59.00,23.00,6.52,-0.36,6.17,...,9.60,32.80,42.50,22.40,6.90,5.20,13.90,19.40,102.10,True
2,2009-10,Utah Jazz,25.60,53.00,29.00,55.00,27.00,5.34,-0.01,5.33,...,10.60,31.60,42.20,26.70,8.20,4.90,15.20,22.70,104.20,True
3,2009-10,San Antonio Spurs,28.70,50.00,32.00,55.00,27.00,5.09,-0.02,5.07,...,10.80,32.00,42.80,22.30,6.30,4.60,13.60,20.40,101.40,True
4,2009-10,Atlanta Hawks,26.50,53.00,29.00,54.00,28.00,4.66,-0.22,4.44,...,11.80,29.90,41.70,21.80,7.20,5.00,12.00,19.90,101.70,True


In [ ]:
px.scatter(merged, x="3PA", y="W", color="Season", hover_name="Team")